<a href="https://colab.research.google.com/github/daemonX10/GSTN/blob/main/pre_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [4]:
X_train = pd.read_csv('/content/X_Train_Data_Input.csv')
y_train = pd.read_csv('/content/Y_Train_Data_Target.csv')


In [5]:
X_test = pd.read_csv('/content/X_Test_Data_Input.csv')
y_test = pd.read_csv('/content/Y_Test_Data_Target.csv')

In [6]:
X_test.drop(columns=['ID'],inplace=True)
y_test.drop(columns=['ID'],inplace=True)
X_train.drop(columns=['ID'],inplace=True)
y_train.drop(columns=['ID'],inplace=True)

In [7]:
for column in X_train.columns:
    mode_value = X_train[column].mode()[0]
    X_train[column].fillna(mode_value, inplace=True)
    test_value= X_test[column].mode()[0]
    X_test[column].fillna(test_value, inplace=True)

print(X_train.isna().sum())

<ipython-input-7-82b790460ce2>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[column].fillna(mode_value, inplace=True)
<ipython-input-7-82b790460ce2>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

Column0     0
Column1     0
Column2     0
Column3     0
Column4     0
Column5     0
Column6     0
Column7     0
Column8     0
Column9     0
Column10    0
Column11    0
Column12    0
Column13    0
Column14    0
Column15    0
Column16    0
Column17    0
Column18    0
Column19    0
Column20    0
Column21    0
dtype: int64


<ipython-input-7-82b790460ce2>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[column].fillna(mode_value, inplace=True)
<ipython-input-7-82b790460ce2>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [8]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [9]:
from sklearn.ensemble import (
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, HistGradientBoostingClassifier,
    RandomForestClassifier, StackingClassifier
)
from sklearn.naive_bayes import (
    BernoulliNB, CategoricalNB, ComplementNB,
    GaussianNB, MultinomialNB
)
from sklearn.tree import (
    DecisionTreeClassifier, ExtraTreeClassifier
)
from sklearn.linear_model import (
    LogisticRegression, LogisticRegressionCV, RidgeClassifier,
    RidgeClassifierCV, SGDClassifier, PassiveAggressiveClassifier,
    Perceptron
)
from sklearn.neighbors import (
    KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
)
from sklearn.svm import (
    LinearSVC, NuSVC, SVC
)
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
)
from sklearn.multioutput import ClassifierChain
from sklearn.semi_supervised import (
    LabelPropagation, LabelSpreading
)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import (
    OneVsOneClassifier, OneVsRestClassifier, OutputCodeClassifier
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from catboost import CatBoostClassifier


In [10]:
from sklearn.metrics import (
    balanced_accuracy_score,
    log_loss,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

def extended_evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)

    # Check if the model has the predict_proba method
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
        # AUC-ROC and Log Loss metrics
        roc_auc = roc_auc_score(y_test, y_proba)
        logloss = log_loss(y_test, y_proba)
    else:
        y_proba = None  # Not applicable for models without predict_proba
        roc_auc = None
        logloss = None

    # Standard metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Additional metrics
    balanced_acc = balanced_accuracy_score(y_test, y_pred)

    # Print results
    print(f"{name} - Accuracy: {accuracy:.4f}")
    print(f"{name} - Precision: {precision:.4f}")
    print(f"{name} - Recall: {recall:.4f}")
    print(f"{name} - F1 Score: {f1:.4f}")

    if roc_auc is not None:
        print(f"{name} - AUC-ROC: {roc_auc:.4f}")
        print(f"{name} - Log Loss: {logloss:.4f}")

    print(f"{name} - Balanced Accuracy: {balanced_acc:.4f}")
    print(f"{name} - Confusion Matrix:\n {conf_matrix}")
    print("\n")


In [ ]:
models = [
    ('AdaBoost', AdaBoostClassifier()),
    ('Bagging', BaggingClassifier()),
    ('BernoulliNB', BernoulliNB()),
    ('CalibratedClassifierCV', CalibratedClassifierCV()),
    ('DecisionTree', DecisionTreeClassifier()),
    ('Dummy', DummyClassifier()),
    ('ExtraTree', ExtraTreeClassifier()),
    ('ExtraTrees', ExtraTreesClassifier()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('HistGradientBoosting', HistGradientBoostingClassifier()),
    ('KNeighbors', KNeighborsClassifier()),
    ('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()),
    ('LogisticRegression', LogisticRegression(max_iter=5000)),
    ('LogisticRegressionCV', LogisticRegressionCV(max_iter=5000)),
    ('MultinomialNB', MultinomialNB()),
    ('NearestCentroid', NearestCentroid()),
    ('OneVsRestClassifier', OneVsRestClassifier(LogisticRegression(max_iter=5000))),
    ('PassiveAggressive', PassiveAggressiveClassifier(max_iter=5000)),
    ('Perceptron', Perceptron(max_iter=5000)),
    ('QuadraticDiscriminantAnalysis', QuadraticDiscriminantAnalysis()),
    ('RadiusNeighbors', RadiusNeighborsClassifier()),
    ('RandomForest', RandomForestClassifier()),
    ('RidgeClassifier', RidgeClassifier()),
    ('RidgeClassifierCV', RidgeClassifierCV()),
    ('SGDClassifier', SGDClassifier()),
    ('StackingClassifier', StackingClassifier(estimators=[('lr', LogisticRegression(max_iter=5000)), ('rf', RandomForestClassifier())])),
    ('XGBClassifier', XGBClassifier()),
    ('CatBoost', CatBoostClassifier(verbose=0)),
    ('ClassifierChain', ClassifierChain(base_estimator=LogisticRegression(max_iter=5000)))
]


model_accuracies = []

for name,model in models:
    model_name = model.__class__.__name__
    print(f"Starting modeling with {model_name}...")
    try:
        model.fit(X_train, y_train)
        extended_evaluate_model(name,model,X_test,y_test)

    except Exception as e:
        print(f"Skipped {model_name} due to error: {str(e)}")


# model_accuracies.sort(key=lambda x: x[1], reverse=True)
# print("\nModel Accuracies:")
# for name, accuracy in model_accuracies:
#     print(f"{name}: {accuracy}")


Starting modeling with AdaBoostClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost - Accuracy: 0.9748
AdaBoost - Precision: 0.8223
AdaBoost - Recall: 0.9348
AdaBoost - F1 Score: 0.8749
AdaBoost - AUC-ROC: 0.9930
AdaBoost - Log Loss: 0.3896
AdaBoost - Balanced Accuracy: 0.9569
AdaBoost - Confusion Matrix:
 [[232049   4985]
 [  1610  23068]]


Starting modeling with BaggingClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:888: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Bagging - Accuracy: 0.9732
Bagging - Precision: 0.8511
Bagging - Recall: 0.8675
Bagging - F1 Score: 0.8592
Bagging - AUC-ROC: 0.9893
Bagging - Log Loss: 0.2047
Bagging - Balanced Accuracy: 0.9258
Bagging - Confusion Matrix:
 [[233289   3745]
 [  3270  21408]]


Starting modeling with BernoulliNB...


...

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoosting - Accuracy: 0.9759
GradientBoosting - Precision: 0.8210
GradientBoosting - Recall: 0.9517
GradientBoosting - F1 Score: 0.8815
GradientBoosting - AUC-ROC: 0.9936
GradientBoosting - Log Loss: 0.0548
GradientBoosting - Balanced Accuracy: 0.9650
GradientBoosting - Confusion Matrix:
 [[231913   5121]
 [  1192  23486]]


Starting modeling with HistGradientBoostingClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoosting - Accuracy: 0.9781
HistGradientBoosting - Precision: 0.8432
HistGradientBoosting - Recall: 0.9432
HistGradientBoosting - F1 Score: 0.8904
HistGradientBoosting - AUC-ROC: 0.9946
HistGradientBoosting - Log Loss: 0.0500
HistGradientBoosting - Balanced Accuracy: 0.9625
HistGradientBoosting - Confusion Matrix:
 [[232706   4328]
 [  1401  23277]]


Starting modeling with KNeighborsClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
